# Looking at Land Cover / Land Use Change (LCLUC) and Population Shifts over 20 years in the Arabian Peninsula
#### Developed by Eqi Luo, Spring 2024.
*this notebook is documented as part of the submission for LRES525 Applied Remote Sensing at Montana State Univerisity Bozeman*

In [1]:
# Import dependices 
import os
import rasterio
import numpy as np
import dask.array as da
import pandas as pd
import matplotlib as plt

## PART I: Calculating LCLU and Population Change Matrix 

The main input data for this  include: 
- [GHS Gridded Population](https://human-settlement.emergency.copernicus.eu/ghs_pop2023.php) resampled to 30m from 100m
- Land Cover Land Use Change from the [GLAD dataset](https://storage.googleapis.com/earthenginepartners-hansen/GLCLU2000-2020/v2/download.html) at 30m resolution.

Before visualizing the relationships between population and LCLUC in the Arabian Peninsula, we need to analyze the raster files and summarize the change information in tabular (.csv) format. The following codes perform the pre-processing:  

### 1.1 Land Cover and Land Use Change
First, we have the layer of land use land cover of UAE for both 2000 and 2020. They are reclassified to 5 lclu classes, and the pixel values represent: 

| Pixel Value | Description |
| :---------- | :---------- |
| **1**    | Built Up |
| **2**     | Cropland |
| **3** | Vegetation |
| **4** | Barren |
| **5** | Water |

 To assess the change at pixel level, we will generate a lclu change layer and also summarize lcluc as a change matrix.

#### <ins>LCLU Change Matrix</ins>
The change matrix summarizes the change information of LCLU classes at pixel level, returning a 5*\5 matrix 

In [2]:
# Function to calculate land cover/land use change matrix

def calculate_transition_block(lclu_2000, lclu_2020):
    """
    Calculates a transition matrix between two land cover/land use (LCLU) classes over different years using Dask arrays.

    Parameters:
        lclu_2000 (Dask Array): The Dask array representing LCLU classes for the year 2000.
        lclu_2020 (Dask Array): The Dask array representing LCLU classes for the year 2020.

    Returns:
        Dask Array: A 5x5 Dask array where each element (i, j) represents the count of pixels transitioning
                    from class i in 2000 to class j in 2020. The classes are indexed from 1 to 5.
    """
    transition_block = da.zeros((5, 5), dtype=int)
    for i in range(1, 6):
        for j in range(1, 6):
            from_class = (lclu_2000 == i)
            to_class = (lclu_2020 == j)
            transition_block[i-1, j-1] += np.sum(from_class & to_class)
    return transition_block

In [3]:
# Define file paths for the raster layers 
lclu_2000_fn = './data/lclu/LCLU_Arabian_Peninsula_2000.tif'
lclu_2020_fn = './data/lclu/LCLU_Arabian_Peninsula_2020.tif'

In [4]:
# Load raster data using Dask for both years
lclu_2000 = da.from_array(rasterio.open(lclu_2000_fn).read(1), chunks='auto')
lclu_2020 = da.from_array(rasterio.open(lclu_2020_fn).read(1), chunks='auto')

In [5]:
# Calculate the transition matrix
transition_matrix = calculate_transition_block(lclu_2000, lclu_2020).compute()
transition_matrix

array([[  60381121,          0,          0,          0,          0],
       [  16735791,  309677981,   16010970,   31810811,     746239],
       [  26324453,   34348501,  551946081,   16875515,    2774280],
       [  42482386,   73707053,   24437368, 6369830526,   16428327],
       [    296980,     418005,     739749,    6599511,  160049390]])

In [6]:
# Define the grid cell size in meters (assuming 30m x 30m cells)
grid_size_meters = 30
grid_size_sqkm = (grid_size_meters / 1000) ** 2

# Convert transition matrix to square kilometers
transition_matrix_sqkm = np.round(transition_matrix * grid_size_sqkm, 2)

In [7]:
# Define class names
class_names = ["Built Up", "Cropland", "Vegetation", "Barren", "Water"]

In [8]:
# Convert transition matrix to DataFrame and format for easier analysis
df = pd.DataFrame(transition_matrix_sqkm, columns=class_names, index=class_names)
melted_df = pd.melt(df.reset_index(), id_vars='index', var_name='class_2020', value_name='area_change')
melted_df.rename(columns={'index': 'class_2000'}, inplace=True)

# Print or save the DataFrame
print(melted_df)

    class_2000  class_2020  area_change
0     Built Up    Built Up     54343.01
1     Cropland    Built Up     15062.21
2   Vegetation    Built Up     23692.01
3       Barren    Built Up     38234.15
4        Water    Built Up       267.28
5     Built Up    Cropland         0.00
6     Cropland    Cropland    278710.18
7   Vegetation    Cropland     30913.65
8       Barren    Cropland     66336.35
9        Water    Cropland       376.20
10    Built Up  Vegetation         0.00
11    Cropland  Vegetation     14409.87
12  Vegetation  Vegetation    496751.47
13      Barren  Vegetation     21993.63
14       Water  Vegetation       665.77
15    Built Up      Barren         0.00
16    Cropland      Barren     28629.73
17  Vegetation      Barren     15187.96
18      Barren      Barren   5732847.47
19       Water      Barren      5939.56
20    Built Up       Water         0.00
21    Cropland       Water       671.62
22  Vegetation       Water      2496.85
23      Barren       Water     14785.49


#### <ins>LCLU Change Layer</ins>
Generate a LCLU change raster file, in which each pixel records the change from 2000 to 2020.

In [9]:
# Define functions

def read_raster_to_dask_array(raster_path, chunks='auto'):
    """
    Reads a raster file into a Dask array, masking out the nodata values.
    
    Parameters:
        raster_path (str): Path to the raster file.
        chunks (str or tuple): Chunk size for Dask array. 'auto' lets Dask decide.
    
    Returns:
        tuple: A tuple containing the Dask array with data and the raster profile.
    """
    with rasterio.open(raster_path) as src:
        data = da.ma.masked_equal(da.from_array(src.read(1), chunks=chunks), src.nodata)
        profile = src.profile.copy()
    return data, profile

def calculate_change_code(lclu_old, lclu_new):
    """
    Calculates the change code between two Dask arrays representing LCLU data.
    
    Parameters:
        lclu_old (Dask Array): The Dask array for the earlier LCLU data.
        lclu_new (Dask Array): The Dask array for the later LCLU data.
    
    Returns:
        Dask Array: The Dask array representing the LCLU change code.
    """
    return (lclu_old * 10 + lclu_new).astype('uint8')

def calculate_lclu_change_layer(path_lclu_2000, path_lclu_2020, output_path):
    """
    Processes LCLU change between two years for a single country and saves the output as raster .tif.
    
    Parameters:
        path_lclu_2000 (str): Path to the LCLU data for the year 2000.
        path_lclu_2020 (str): Path to the LCLU data for the year 2020.
        output_path (str): Path where the output change file will be saved.
    """
    
    # Read the raster datasets into Dask arrays, masking nodata values
    lclu_2000, profile_2000 = read_raster_to_dask_array(path_lclu_2000)
    lclu_2020, profile_2020 = read_raster_to_dask_array(path_lclu_2020)
    
    # Check if the nodata values are consistent between the two datasets
    if profile_2000['nodata'] != profile_2020['nodata']:
        print("Nodata values do not match.")
        return
    
    # Calculate the change codes using the Dask arrays
    change_matrix_code = calculate_change_code(lclu_2000, lclu_2020)
    
    # Prepare the output raster's profile
    profile_2000.update(dtype='uint8', count=1, compress='DEFLATE')
    
    # Write the LCLUC change layer to a new raster file
    with rasterio.open(output_path, 'w', **profile_2000) as dst:
        dst.write(change_matrix_code.compute(), 1)

In [10]:
# specify the raster output path
output_path = './data/lclu/LCLUC_Arabian_Peninsula_2000-2020.tif'

# generate the lcluc raster layer
calculate_lclu_change_layer(lclu_2000_fn, lclu_2020_fn, output_path)

## 1.2 Population Change
Next, let's calculate the population change from 2000 to 2020 at the pixel level, and also its distribution on different LCLUC.

#### <ins> Population Change Raster </ins>


In [11]:
# Define file paths for the population layers 
pop_2000_fn = './data/population/ghs_pop_2000_Arabian_Peninsula_30m.tif'
pop_2020_fn = './data/population/ghs_pop_2020_Arabian_Peninsula_30m.tif'

# Define the output population change layer
output_path = './data/population/ghs_pop_change_2000-2020_Arabian_Peninsula.tif'

In [12]:
def calculate_pop_change_layer(path_pop_2000, path_pop_2020, output_path):
    """
    Processes population change between two years for a single country and saves the output as raster .tif.
    
    Parameters:
        path_pop_2000 (str): Path to the population data for the year 2000.
        path_pop_2020 (str): Path to the population data for the year 2020.
        output_path (str): Path where the output change file will be saved.
    """
    
    # Read the raster datasets into Dask arrays, masking nodata values
    pop_2000, profile_2000 = read_raster_to_dask_array(path_pop_2000)
    pop_2020, profile_2020 = read_raster_to_dask_array(path_pop_2020)
    
    # Check if the nodata values are consistent between the two datasets
    if profile_2000['nodata'] != profile_2020['nodata']:
        print("Nodata values do not match.")
        return
    
    # calculate the population change, directly specifying the data type
    pop_change = (pop_2020 - pop_2000).astype('float32')
    
    # Prepare the output raster's profile
    profile_2000.update(dtype='float32', count=1, compress='DEFLATE')
    
    # Write the LCLUC change layer to a new raster file
    with rasterio.open(output_path, 'w', **profile_2000) as dst:
        dst.write(pop_change.compute(), 1)

In [13]:
# calculate the population change
calculate_pop_change_layer(pop_2000_fn, pop_2020_fn, output_path)

#### <ins> Population Change Distribution on LCLUC </ins>
Now, after we have the LCLUC and Population Change layer, now we can calculate the population change distribution on each LCLUC classes from 2000 to 2020.

In [14]:
def calculate_popchange_by_lcluc(lclu_change_fn, pop_change_fn, chunks='auto'):
    """
    Calculates the total population change associated with each land cover/land use change class.

    Parameters:
        lclu_change_fn (str): File path to the land cover/land use change raster.
        pop_change_fn (str): File path to the population change raster.
        chunks (str or tuple, optional): Chunk size for Dask arrays. Default is 'auto'.

    Returns:
        dict: A dictionary where keys are LCLU change classes and values are the total population change.
    """
    
    # Open the LCLU change raster
    #with rasterio.open(lclu_change_fn) as lclu_change_src:
        #lclu_change_data = da.ma.masked_equal(da.from_array(lclu_change_src.read(1), chunks=chunks), lclu_change_src.nodatavals[0])
    
    # Open the LCLU change raster
    lclu_change_data, profile_lclu = read_raster_to_dask_array(lclu_change_fn)

    # Open the population change raster
    #with rasterio.open(pop_change_fn) as pop_change_src:
        #pop_change_data = da.ma.masked_equal(da.from_array(pop_change_src.read(1), chunks=chunks), pop_change_src.nodatavals[0])
    
    # Open the population change raster
    pop_change_data, profile_pop = read_raster_to_dask_array(pop_change_fn)

    # Get unique LCLU change classes from the matrix
    unique_lclu_changes = da.unique(lclu_change_data).compute()

    # Initialize a dictionary to hold the total population change for each LCLU change class
    population_change_distribution = {lclu_change: 0 for lclu_change in unique_lclu_changes}

    # Calculate the total population change for each LCLU change class
    for lclu_change in unique_lclu_changes:
        mask = lclu_change_data == lclu_change  # Create a mask for the current LCLU change class
        # Use the mask to select the corresponding population changes and sum them up
        population_change_distribution[lclu_change] = pop_change_data[mask].sum().compute()

    return population_change_distribution


In [15]:
# specify the path
lcluc_fn = './data/lclu/LCLUC_Arabian_Peninsula_2000-2020.tif'
popc_fn = './data/population/ghs_pop_change_2000-2020_Arabian_Peninsula.tif'

In [16]:
# run the analysis to calculate pop distribution on lcluc
pop_by_lcluc = calculate_popchange_by_lcluc(lcluc_fn, popc_fn)

/home/m63q478/.conda/envs/eqi-research/lib/python3.11/site-packages/dask/array/core.py:1988: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  self, index2 = slice_with_bool_dask_array(self, index2)
/home/m63q478/.conda/envs/eqi-research/lib/python3.11/site-packages/dask/array/core.py:1988: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To av

In [17]:
pop_by_lcluc

{11: 36670160.0,
 21: 4058121.2,
 22: 1009902.2,
 23: 210865.92,
 24: 329545.56,
 25: 4881.5376,
 31: 9238367.0,
 32: 161893.73,
 33: 3359766.2,
 34: 585251.5,
 35: 8233.131,
 41: 31553784.0,
 42: 122450.92,
 43: 255800.92,
 44: 10315752.0,
 45: 25723.996,
 51: 93470.66,
 52: 801.1794,
 53: 1422.0834,
 54: 20597.023,
 55: 77529.82,
 255: masked}

In [18]:
# Create a pandas DataFrame to store the results
df_pop_change = pd.DataFrame(list(pop_by_lcluc.items()), columns=['LCLUC_Class', 'Pop_Change'])
df_pop_change

,LCLUC_Class,Pop_Change
0,11,36670160.0
1,21,4058121.25
2,22,1009902.1875
3,23,210865.921875
4,24,329545.5625
5,25,4881.537598
6,31,9238367.0
7,32,161893.734375
8,33,3359766.25
9,34,585251.5


## PART II: Integrating LCLUC and Population Change Data 

Now that we have both the LCLUC info and Population change info, we can merge them into one single .csv file for our data analysis later.

In [19]:
# Mapping from class description to numerical code
class_mapping = {'Built Up': 1, 'Cropland': 2, 'Vegetation': 3, 'Barren': 4, 'Water': 5}

In [20]:
# Generate LCLUC codes for the area change DataFrame
melted_df['LCLUC_Code'] = melted_df.apply(
    lambda row: 10 * class_mapping[row['class_2000']] + class_mapping[row['class_2020']], axis=1)

melted_df

,class_2000,class_2020,area_change,LCLUC_Code
0,Built Up,Built Up,54343.01,11
1,Cropland,Built Up,15062.21,21
2,Vegetation,Built Up,23692.01,31
3,Barren,Built Up,38234.15,41
4,Water,Built Up,267.28,51
5,Built Up,Cropland,0.00,12
6,Cropland,Cropland,278710.18,22
7,Vegetation,Cropland,30913.65,32
8,Barren,Cropland,66336.35,42
9,Water,Cropland,376.20,52


In [21]:
# Merge the population data into the area change DataFrame using LCLUC codes
df_combined = melted_df.merge(df_pop_change, left_on='LCLUC_Code', right_on='LCLUC_Class', how='left')

df_combined

,class_2000,class_2020,area_change,LCLUC_Code,LCLUC_Class,Pop_Change
0,Built Up,Built Up,54343.01,11,11.0,36670160.0
1,Cropland,Built Up,15062.21,21,21.0,4058121.25
2,Vegetation,Built Up,23692.01,31,31.0,9238367.0
3,Barren,Built Up,38234.15,41,41.0,31553784.0
4,Water,Built Up,267.28,51,51.0,93470.65625
5,Built Up,Cropland,0.00,12,NaN,NaN
6,Cropland,Cropland,278710.18,22,22.0,1009902.1875
7,Vegetation,Cropland,30913.65,32,32.0,161893.734375
8,Barren,Cropland,66336.35,42,42.0,122450.921875
9,Water,Cropland,376.20,52,52.0,801.179382


In [22]:
# Drop the 'LCLUC_Class' column
df_combined.drop('LCLUC_Class', axis=1, inplace=True)

# Reorder columns to make 'LCLUC_Code' the third column
columns = df_combined.columns.tolist()
columns.insert(2, columns.pop(columns.index('LCLUC_Code')))
df_clean = df_combined[columns].copy()

# Fill NaN values with 0 in the 'Pop_Change' column
df_clean['Pop_Change'].fillna(0, inplace=True)

print(df_combined)

    class_2000  class_2020  area_change  LCLUC_Code     Pop_Change
0     Built Up    Built Up     54343.01          11     36670160.0
1     Cropland    Built Up     15062.21          21     4058121.25
2   Vegetation    Built Up     23692.01          31      9238367.0
3       Barren    Built Up     38234.15          41     31553784.0
4        Water    Built Up       267.28          51    93470.65625
5     Built Up    Cropland         0.00          12            NaN
6     Cropland    Cropland    278710.18          22   1009902.1875
7   Vegetation    Cropland     30913.65          32  161893.734375
8       Barren    Cropland     66336.35          42  122450.921875
9        Water    Cropland       376.20          52     801.179382
10    Built Up  Vegetation         0.00          13            NaN
11    Cropland  Vegetation     14409.87          23  210865.921875
12  Vegetation  Vegetation    496751.47          33     3359766.25
13      Barren  Vegetation     21993.63          43  255800.92

In [23]:
df_combined.to_csv('LCLUC_Pop_change_data_2000-2020_Arabian_Peninsula.csv')